In [37]:
import pandas as pd
import numpy as np

from sklearn.linear_model import SGDRegressor # Регрессия
from sklearn.linear_model import LogisticRegression # Классификация
from sklearn.tree import DecisionTreeClassifier # Классификация при помощи решающего дерева

from sklearn.metrics import accuracy_score # Метрика accuracy
from sklearn.metrics import mean_absolute_error # Метрика mae для регрессии
from sklearn.metrics import mean_squared_error # Метрика для mse регрессии
from sklearn.metrics import f1_score # Метрика f1 score

from sklearn.datasets import load_iris # Датасет для классификации и решающих деревьев
from sklearn.datasets import load_diabetes # Датасет для регрессии

from sklearn.model_selection import train_test_split # Разбиение на выборки

from sklearn.model_selection import GridSearchCV # Сеточный метод поиска параметров
from sklearn.model_selection import KFold # Разбиение на фолды
from sklearn.pipeline import make_pipeline # Разбиение на фолды

from sklearn.preprocessing import PolynomialFeatures, StandardScaler # Предварительная обработка данных

# Загрузка тестовых датасетов

In [38]:
X_class, Y_class = load_iris(as_frame=True)['data'], load_iris(as_frame=True)['target']

display(X_class)
display(Y_class)

X_reg, Y_reg = load_diabetes(as_frame=True)['data'], load_diabetes(as_frame=True)['target']

display(X_reg)
display(Y_reg)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


,target
0,0
1,0
2,0
3,0
4,0
...,...
145,2
146,2
147,2
148,2


,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641
...,...,...,...,...,...,...,...,...,...,...
437,0.041708,0.050680,0.019662,0.059744,-0.005697,-0.002566,-0.028674,-0.002592,0.031193,0.007207
438,-0.005515,0.050680,-0.015906,-0.067642,0.049341,0.079165,-0.028674,0.034309,-0.018114,0.044485
439,0.041708,0.050680,-0.015906,0.017293,-0.037344,-0.013840,-0.024993,-0.011080,-0.046883,0.015491
440,-0.045472,-0.044642,0.039062,0.001215,0.016318,0.015283,-0.028674,0.026560,0.044529,-0.025930


,target
0,151.0
1,75.0
2,141.0
3,206.0
4,135.0
...,...
437,178.0
438,104.0
439,132.0
440,220.0


# Разбиение на выборки

In [31]:
# Классификация
X_train_class, X_test_class, Y_train_class, Y_test_class \
    = train_test_split(X_class, Y_class, test_size = 0.2, random_state=13)
X_train_class, X_val_class, Y_train_class, Y_val_class \
    = train_test_split(X_train_class, Y_train_class, test_size = 0.25, random_state=13)

# Регрессия
X_train_reg, X_test_reg, Y_train_reg, Y_test_reg \
    = train_test_split(X_reg, Y_reg, test_size = 0.2, random_state=13)
X_train_reg, X_val_reg, Y_train_reg, Y_val_reg \
    = train_test_split(X_train_reg, Y_train_reg, test_size = 0.25, random_state=13)

# Просмотр документации

In [4]:
# print(DecisionTreeClassifier.__doc__)


# Обучение

In [41]:
model_class = LogisticRegression()
model_class.fit(X_train_class, Y_train_class)

model_reg = SGDRegressor()
model_reg.fit(X_train_reg, Y_train_reg)

model_tree = DecisionTreeClassifier(max_depth=20)
model_tree.fit(X_train_class, Y_train_class)


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1608: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


DecisionTreeClassifier(max_depth=20)

# Предсказание

In [42]:
pred_test_class = model_class.predict(X_test_class)
pred_test_reg = model_reg.predict(X_test_reg)
pred_test_tree = model_tree.predict(X_test_class)

In [45]:
Y_test_class.to_numpy(), pred_test_class

(array([1, 1, 0, 2, 2, 0, 2, 2, 0, 1, 2, 2, 1, 0, 2, 0, 2, 2, 1, 0, 1, 0,
        0, 2, 1, 2, 0, 2, 1, 2]),
 array([1, 1, 0, 2, 2, 0, 2, 2, 0, 1, 1, 1, 1, 0, 2, 0, 2, 2, 1, 0, 1, 0,
        0, 2, 1, 2, 0, 2, 1, 2]))

# Расчет метрик

In [46]:
print(accuracy_score(Y_test_class, pred_test_class))
print(mean_absolute_error(Y_test_reg, pred_test_reg))
print(f1_score(Y_test_class, pred_test_tree, average='weighted'))

0.9333333333333333
46.96389687317487
1.0


# Подбор параметров моделей

In [49]:
list_param = {'max_depth': [2, 10, 20],
              'max_features': ['log2'],
              'min_samples_leaf': range(1,4),
              'min_samples_split': range(2, 5),
              'criterion': ['gini', 'entropy']}

model_tree = DecisionTreeClassifier()

gr_search = GridSearchCV(model_tree, list_param,
                         cv=KFold(n_splits=5, shuffle=True, random_state=13), verbose=1)

gr_search.fit(X_train_class, Y_train_class)

print("\nBest parameters\n")
for j in gr_search.best_params_:
    print(f"{j}: {gr_search.best_params_[j]}")
print("\n")
model_tree = gr_search.best_estimator_
model_tree.fit(X_train_class, Y_train_class)

pred_train_tree = model_tree.predict(X_train_class)
pred_val_tree = model_tree.predict(X_val_class)
pred_test_tree = model_tree.predict(X_test_class)


print(f1_score(Y_train_class, pred_train_tree, average='weighted'))
print(f1_score(Y_val_class, pred_val_tree, average='weighted'))
print(f1_score(Y_test_class, pred_test_tree, average='weighted'))

Fitting 5 folds for each of 54 candidates, totalling 270 fits

Best parameters

criterion: gini
max_depth: 2
max_features: log2
min_samples_leaf: 1
min_samples_split: 3


0.9665403672398718
0.9333333333333333
0.9669803921568628


In [51]:
model = make_pipeline(PolynomialFeatures(), StandardScaler(with_mean=False), SGDRegressor(random_state=43))
list_param = {'polynomialfeatures__degree': [i for i in range(2, 7)],
              'sgdregressor__alpha': [0.1, 0.2, 0.0001]}


gr_search = GridSearchCV(model, list_param , scoring='neg_mean_absolute_error',
                         cv=KFold(n_splits=5, shuffle=True, random_state=13), verbose=4).fit(X_train_reg, Y_train_reg)
print("Best parameters\n")
for j in gr_search.best_params_:
    print(f"{j}: {gr_search.best_params_[j]}")
print("\n")
model_lr = gr_search.best_estimator_
model_lr.fit(X_train_reg,
    Y_train_reg)

pred_train_lr = model_lr.predict(X_train_reg)
pred_test_lr = model_lr.predict(X_test_reg)


print(mean_absolute_error(Y_train_reg, pred_train_lr))
print(mean_absolute_error(Y_test_reg, pred_test_lr))

Fitting 5 folds for each of 15 candidates, totalling 75 fits
[CV 1/5] END polynomialfeatures__degree=2, sgdregressor__alpha=0.1;, score=-52.385 total time=   0.0s
[CV 2/5] END polynomialfeatures__degree=2, sgdregressor__alpha=0.1;, score=-41.863 total time=   0.0s
[CV 3/5] END polynomialfeatures__degree=2, sgdregressor__alpha=0.1;, score=-46.942 total time=   0.0s
[CV 4/5] END polynomialfeatures__degree=2, sgdregressor__alpha=0.1;, score=-41.934 total time=   0.0s
[CV 5/5] END polynomialfeatures__degree=2, sgdregressor__alpha=0.1;, score=-46.399 total time=   0.0s
[CV 1/5] END polynomialfeatures__degree=2, sgdregressor__alpha=0.2;, score=-49.244 total time=   0.0s
[CV 2/5] END polynomialfeatures__degree=2, sgdregressor__alpha=0.2;, score=-41.743 total time=   0.0s
[CV 3/5] END polynomialfeatures__degree=2, sgdregressor__alpha=0.2;, score=-48.022 total time=   0.0s
[CV 4/5] END polynomialfeatures__degree=2, sgdregressor__alpha=0.2;, score=-42.678 total time=   0.0s
[CV 5/5] END polynomi